# Quantitative Momentum Investment Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

P.s. `hqm` stands for `high-quality momentum`.

## Library Imports

In [1]:
import pandas as pd 
import numpy as np 
import requests
import math
from scipy import stats
import warnings
from api import pub
warnings.filterwarnings('ignore')

## Importing list of stocks from wikipedia

In [2]:
tickers = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
stocks = pd.DataFrame({'Ticker':tickers['Symbol']})
stocks

,Ticker
0,MMM
1,AOS
2,ABT
3,ABBV
4,ACN
...,...
498,YUM
499,ZBRA
500,ZBH
501,ZION


## Filling out the dataframe with API data

In [3]:
def chunks(lst, n): # converts list to list of chunks
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [4]:
# Dividing tickers by 100 to iterate batch requests

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

symbol_strings

['MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,APD,ABNB,AKAM,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,AME,AMGN,APH,ADI,ANSS,AON,APA,AAPL,AMAT,APTV,ACGL,ADM,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,AXON,BKR,BALL,BAC,BK,BBWI,BAX,BDX,BRK.B,BBY,BIO,TECH,BIIB,BLK,BX,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,BF.B,BLDR,BG,CDNS,CZR,CPT,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,COR,CNC,CNP,CDAY',
 'CF,CHRW,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CMA,CAG,COP,ED,STZ,CEG,COO,CPRT,GLW,CTVA,CSGP,COST,CTRA,CCI,CSX,CMI,CVS,DHR,DRI,DVA,DE,DAL,XRAY,DVN,DXCM,FANG,DLR,DFS,DG,DLTR,D,DPZ,DOV,DOW,DHI,DTE,DUK,DD,EMN,ETN,EBAY,ECL,EIX,EW,EA,ELV,LLY,EMR,ENPH,ETR,EOG,EPAM,EQT,EFX,EQIX,EQR,ESS,EL,ETSY,EG,EVRG,ES,EXC,EXPE,EXPD,EXR,XOM,FFIV,FDS,FICO,FAST,FRT,FDX,FIS,FITB,FSLR,FE,FI',
 'FLT,FMC,F,FTNT,FTV,FOXA,FOX,BEN,FCX,GRMN,IT,GEHC,GEN,GNRC,GD,GE,GIS,GM,GPC,GILD,GPN,GL,GS,HAL,HIG,HAS,HCA,PEAK,HSIC,HSY,HES,HPE,HLT,HOLX,HD,HON,HRL,HST,HW

In [5]:
# Here you can see columns which affect quality

hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    "One-Month Price Return",
    'One-Month Return Percentile'

]

In [6]:
hqm_df = pd.DataFrame(columns = hqm_columns)

# Batch API request to IEX CLOUD
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE,advanced_stats/{symbol_string}/?token={pub}'
    data = requests.get(batch_api_call_url).json()
    half = int(len(data)/2)
    quote = data[:half]
    adv_stats = data[half:]

    for s_index, symbol in enumerate(symbol_string.split(',')):
        new_col = {'Ticker': symbol, 'Price': quote[s_index]['latestPrice'], 
                   'Number of Shares to Buy': 'N/A', 
                   'One-Year Price Return': adv_stats[s_index]['year1ChangePercent'],
                   'One-Year Return Percentile': 'N/A', 'Six-Month Price Return': adv_stats[s_index]['month6ChangePercent'], 
                   'Six-Month Return Percentile': 'N/A',
                   'Three-Month Price Return': adv_stats[s_index]['month3ChangePercent'], 
                   'Three-Month Return Percentile': 'N/A', 
                   "One-Month Price Return": adv_stats[s_index]['month1ChangePercent'],
                   'One-Month Return Percentile': 'N/A'}
        hqm_df.loc[len(hqm_df)] = new_col
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,MMM,109.140,N/A,-0.040809,N/A,0.140881,N/A,0.181801,N/A,0.120223,N/A
1,AOS,82.335,N/A,0.450893,N/A,0.146908,N/A,0.261262,N/A,0.081622,N/A
2,ABT,110.470,N/A,0.031466,N/A,0.029105,N/A,0.155634,N/A,0.068932,N/A
3,ABBV,154.895,N/A,-0.011370,N/A,0.192794,N/A,0.021552,N/A,0.113524,N/A
4,ACN,352.880,N/A,0.354798,N/A,0.185389,N/A,0.130121,N/A,0.064344,N/A
...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,130.920,N/A,0.026188,N/A,-0.016423,N/A,0.077709,N/A,0.024669,N/A
499,ZBRA,275.425,N/A,0.098765,N/A,-0.024822,N/A,0.220796,N/A,0.191317,N/A
500,ZBH,121.660,N/A,-0.040934,N/A,-0.160036,N/A,0.098187,N/A,0.066159,N/A
501,ZION,44.450,N/A,-0.043032,N/A,0.669398,N/A,0.358205,N/A,0.293895,N/A


## Feature engineering

In [7]:
# Replacing companies with nan in "...Price Return" rows

time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]
for period in time_periods:
    hqm_df = hqm_df.dropna(subset=[f'{period} Price Return'])

In [8]:
# Calculating values for "...Return Percentile" columns

for row in hqm_df.index:
    for period in time_periods:
        percentile_col = f'{period} Return Percentile'
        change_col = f'{period} Price Return'
        hqm_df.loc[row, percentile_col] = stats.percentileofscore(hqm_df[change_col],
                                                                  hqm_df.loc[row, change_col])


hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,MMM,109.140,N/A,-0.040809,24.103586,0.140881,64.342629,0.181801,68.326693,0.120223,69.920319
1,AOS,82.335,N/A,0.450893,85.657371,0.146908,66.334661,0.261262,84.661355,0.081622,51.792829
2,ABT,110.470,N/A,0.031466,36.055777,0.029105,35.258964,0.155634,60.557769,0.068932,45.418327
3,ABBV,154.895,N/A,-0.011370,29.282869,0.192794,75.896414,0.021552,19.123506,0.113524,66.533865
4,ACN,352.880,N/A,0.354798,78.486056,0.185389,74.103586,0.130121,52.788845,0.064344,43.027888
...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,130.920,N/A,0.026188,35.458167,-0.016423,23.904382,0.077709,35.657371,0.024669,25.697211
499,ZBRA,275.425,N/A,0.098765,46.613546,-0.024822,21.713147,0.220796,77.49004,0.191317,87.450199
500,ZBH,121.660,N/A,-0.040934,23.904382,-0.160036,4.98008,0.098187,42.430279,0.066159,44.223108
501,ZION,44.450,N/A,-0.043032,23.505976,0.669398,100.0,0.358205,96.215139,0.293895,97.808765


In [9]:
# Evaluating High Quality Momentum score by calculating Percentile columns mean

HQM_score_col = []
for row in hqm_df.index:
    momentum_percentiles = []
    for period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{period} Return Percentile'])
    HQM_score_col.append(np.mean(momentum_percentiles))
hqm_df['HQM score'] = HQM_score_col
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM score
0,MMM,109.140,N/A,-0.040809,24.103586,0.140881,64.342629,0.181801,68.326693,0.120223,69.920319,56.673307
1,AOS,82.335,N/A,0.450893,85.657371,0.146908,66.334661,0.261262,84.661355,0.081622,51.792829,72.111554
2,ABT,110.470,N/A,0.031466,36.055777,0.029105,35.258964,0.155634,60.557769,0.068932,45.418327,44.322709
3,ABBV,154.895,N/A,-0.011370,29.282869,0.192794,75.896414,0.021552,19.123506,0.113524,66.533865,47.709163
4,ACN,352.880,N/A,0.354798,78.486056,0.185389,74.103586,0.130121,52.788845,0.064344,43.027888,62.101594
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,130.920,N/A,0.026188,35.458167,-0.016423,23.904382,0.077709,35.657371,0.024669,25.697211,30.179283
499,ZBRA,275.425,N/A,0.098765,46.613546,-0.024822,21.713147,0.220796,77.49004,0.191317,87.450199,58.316733
500,ZBH,121.660,N/A,-0.040934,23.904382,-0.160036,4.98008,0.098187,42.430279,0.066159,44.223108,28.884462
501,ZION,44.450,N/A,-0.043032,23.505976,0.669398,100.0,0.358205,96.215139,0.293895,97.808765,79.382470


## Taking top 50 companies with the best HQM score

In [10]:
hqm_df_sorted = hqm_df.sort_values('HQM score',ascending=False)[:50]
hqm_df_sorted.reset_index(inplace=True)
del hqm_df_sorted['index']
hqm_df_sorted

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM score
0,BLDR,168.880,N/A,1.617959,99.40239,0.277794,89.442231,0.383181,97.211155,0.248975,95.61753,95.418327
1,AVGO,1125.080,N/A,1.101492,98.007968,0.346932,96.015936,0.391446,97.609562,0.196063,88.446215,95.019920
2,RCL,129.500,N/A,1.656851,99.601594,0.280883,89.840637,0.412788,98.007968,0.220708,92.231076,94.920319
3,AMD,149.740,N/A,1.308677,98.605578,0.323218,93.824701,0.489446,99.800797,0.190950,87.250996,94.870518
4,PHM,103.790,N/A,1.286281,98.406375,0.326372,94.422311,0.422584,98.406375,0.184543,86.454183,94.422311
5,INTC,50.370,N/A,1.024561,97.609562,0.509174,99.003984,0.476289,99.601594,0.151543,81.075697,94.322709
6,URI,581.155,N/A,0.636686,92.430279,0.376408,96.613546,0.301645,90.239044,0.247032,95.418327,93.675299
7,BX,133.205,N/A,0.863877,96.414343,0.475972,98.605578,0.255461,83.665339,0.242248,95.219124,93.476096
8,UBER,63.295,N/A,1.593443,99.203187,0.443760,98.406375,0.401861,97.808765,0.128791,73.505976,92.231076
9,EXPE,153.770,N/A,0.770448,95.418327,0.417800,97.808765,0.528678,100.0,0.127667,72.908367,91.533865


## Function which shows minimum budget for buying all positions in dataframe

In [11]:
def min_budget(df):
    minimum = math.ceil(np.sum(df['Price']))
    print(f'To own the above portfolio with 1 stock per company you need at least ${minimum}')
    
min_budget(hqm_df_sorted)

To own the above portfolio with 1 stock per company you need at least $15410


## Calculating the number of shares with the same monetary amount per company

In [12]:
def calc_num_of_shares(df):

    while True:
        try:
            whole_amount = int(input('Input your portfolio size in dollars -> '))
            break
        except:
            print('Do not input text, only numbers!\n')
    
    amount_on_ticker = whole_amount/len(df.index)
    df['Number of Shares to Buy'] = [math.floor(amount_on_ticker/price) for price in df['Price']]


calc_num_of_shares(hqm_df_sorted)
hqm_df_sorted

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM score
0,BLDR,168.880,118,1.617959,99.40239,0.277794,89.442231,0.383181,97.211155,0.248975,95.61753,95.418327
1,AVGO,1125.080,17,1.101492,98.007968,0.346932,96.015936,0.391446,97.609562,0.196063,88.446215,95.019920
2,RCL,129.500,154,1.656851,99.601594,0.280883,89.840637,0.412788,98.007968,0.220708,92.231076,94.920319
3,AMD,149.740,133,1.308677,98.605578,0.323218,93.824701,0.489446,99.800797,0.190950,87.250996,94.870518
4,PHM,103.790,192,1.286281,98.406375,0.326372,94.422311,0.422584,98.406375,0.184543,86.454183,94.422311
5,INTC,50.370,397,1.024561,97.609562,0.509174,99.003984,0.476289,99.601594,0.151543,81.075697,94.322709
6,URI,581.155,34,0.636686,92.430279,0.376408,96.613546,0.301645,90.239044,0.247032,95.418327,93.675299
7,BX,133.205,150,0.863877,96.414343,0.475972,98.605578,0.255461,83.665339,0.242248,95.219124,93.476096
8,UBER,63.295,315,1.593443,99.203187,0.443760,98.406375,0.401861,97.808765,0.128791,73.505976,92.231076
9,EXPE,153.770,130,0.770448,95.418327,0.417800,97.808765,0.528678,100.0,0.127667,72.908367,91.533865


## Saving dataframe to excel

In [13]:
# Create ExcelWriter object
writer = pd.ExcelWriter('data/momentum_strategy.xlsx', engine='xlsxwriter')

# Write DataFrame to Excel
hqm_df_sorted.to_excel(writer, sheet_name='Momentum Strategy', index=False)


background_color = '#d3d3d3' 

string_format = writer.book.add_format(
    {
        'bg_color':background_color,
        'border':1
    }
)
dollar_format_long = writer.book.add_format(
    {   
        'num_format':'$0.00',
        'bg_color':background_color,
        'border':1
    }
)
dollar_format_short = writer.book.add_format(
    {   
        'num_format':'$0',
        'bg_color':background_color,
        'border':1
    }
)
integer_format = writer.book.add_format(
    {   
        'num_format':'0',
        'bg_color':background_color,
        'border':1
    }
)

percent_format = writer.book.add_format(
    {   
        'num_format':'0.0%',
        'bg_color':background_color,
        'border':1
    }
)

percentile_format = writer.book.add_format(
    {   
        'num_format':'0"th"',
        'bg_color':background_color,
        'border':1
    }
) 

float_format = writer.book.add_format(
    {   
        'num_format':'0.00',
        'bg_color':background_color,
        'border':1
    }
) 

header_format = writer.book.add_format(
    {   
        'bg_color':'#008080',
        'border':1,
        'bold':True
    }
)



latin_letters_dict = {num: chr(num + 65) for num in range(26)}

col_format_dict = {
    'Ticker':string_format,
    'Price':dollar_format_long,
    'Number of Shares to Buy':integer_format,
    'One-Year Price Return':percent_format,
    'One-Year Return Percentile':percentile_format,
    'Six-Month Price Return':percent_format,
    'Six-Month Return Percentile':percentile_format,
    'Three-Month Price Return':percent_format,
    'Three-Month Return Percentile':percentile_format,
    "One-Month Price Return":percent_format,
    'One-Month Return Percentile':percentile_format,
    'HQM score': float_format
}

for i, col in enumerate(hqm_df_sorted.columns):
    letter = latin_letters_dict[i]
    writer.sheets['Momentum Strategy'].write(f'{letter}1', col, header_format)
    writer.sheets['Momentum Strategy'].set_column(f'{letter}:{letter}', 23)
    writer.sheets['Momentum Strategy'].conditional_format(1, i, len(hqm_df_sorted)+1, i, 
                                                          {'type': 'no_blanks', 'format': col_format_dict[col]})

    
writer.close()